# Simu-Learn - Autonomous Multi-Agent Learning System

## Welcome to the autonomous agentic simulation for a virtual learner

```
Main: Orchestrator → CurriculumDesigner → (for each topic: Tutor → QuizWorkflow)
QuizWorkflow: QuizMaster → Learner → Reviewer (repeated N times)
```

In [ ]:
%pip install -qU ipywidgets

In [1]:
import os
import random

from openai import OpenAI

## Utilities

In [2]:
def _get_secret_from_environment(name: str) -> str:
  secret = os.environ.get(name, '').strip()

  if secret:
      print(f'✅ Environment Variable: Found {name}')
      return secret

  print(f'❌ Environment Variable : {name} is not set')

  try:
    from google.colab import userdata

    secret = userdata.get(name)
    print(f'✅ Google Colab: Found {name}')
  except Exception as e:
    print(f'❌ Google Colab: {e}')

  return secret.strip()


def _get_secret_from_user(name: str) -> str:
  from getpass import getpass

  return getpass(f'Enter the secret value for {name}: ')

def get_secret(name: str) -> str:
  secret = _get_secret_from_environment(name)
  if not secret:
    secret = _get_secret_from_user(name)

  return secret

### Setup LLM Provider (Ollama)

🔑 Go to https://ollama.com/ to register for a **FREE** api key, if you need one

In [4]:
MODEL = 'gpt-oss:20b-cloud'

api_key = get_secret('OLLAMA_API_KEY')
client = OpenAI(api_key=api_key, base_url='https://ollama.com/v1')


[model.id for model in client.models.list()]

✅ Environment Variable: Found OLLAMA_API_KEY


['glm-4.6',
 'kimi-k2:1t',
 'qwen3-coder:480b',
 'deepseek-v3.1:671b',
 'gpt-oss:120b',
 'gpt-oss:20b',
 'qwen3-vl:235b-instruct',
 'qwen3-vl:235b',
 'minimax-m2']

## Agents

In [ ]:
import time
from typing import Any
from dataclasses import dataclass, field
from datetime import datetime, timedelta


@dataclass
class State:
    educational_level: str
    subject: str
    topics: list[str]
    topic: str
    mastery: dict[str, float]
    session_start: datetime
    history: list[Any]
    quiz: dict[str, str]
    learner_answer: str = field(default='')
    num_questions_per_topic: int = field(default=2)
    run_duration_minutes: int = field(default=5)


@dataclass
class AgentResponse:
    agent: str
    message: str
    current_state: State
    metadata: dict = field(default_factory=dict)


def add_message(role, text):
    timestamp = datetime.now().strftime('%H:%M:%S')
    print(f'\n[{timestamp}] {role}: {text}\n')
    time.sleep(1)

### Agents Definitions

In [ ]:

from abc import ABC, abstractmethod
from typing import override
import json
import copy


class Agent(ABC):
    @abstractmethod
    def run(self, state: State) -> AgentResponse:
        ...

    def copy_state(self, state: State) -> State:
        return copy.deepcopy(state)


class CurriculumDesigner(Agent):

    @override
    def run(self, state: State) -> AgentResponse:
        prompt = f'''
        You are an experienced educationist in {state.subject}, tasked
        to generate 5 topics for a learner at the {state.educational_level} level as a JSON output.

        strictly produce **ONLY JSON output** with the list of topics:
        Example:
        ["topic 1", "topic 2"]
        '''

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{'role': 'user', 'content': prompt}],
            response_format={"type": "json_object"},
        )
        msg = resp.choices[0].message.content
        topics = json.loads(msg)

        current_state = self.copy_state(state)
        current_state.topics = topics

        return AgentResponse(
            agent='📚 CurriculumDesigner',
            message=f'{len(topics)} topics. {str(topics)}',
            current_state=current_state,
        )


class Tutor(Agent):

    @override
    def run(self, state: State) -> AgentResponse:
        prompt = f'''
        You are a tutor in {state.subject}.
        Explain {state.topic} a learner at the {state.educational_level} level.
        in not more than 2 short paragraphs
        '''

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{'role': 'user', 'content': prompt}],
        )
        msg = resp.choices[0].message.content

        current_state = self.copy_state(state)

        return AgentResponse(
            agent='📘 Tutor',
            message=msg,
            current_state=current_state,
        )


class QuizMaster(Agent):

    @override
    def run(self, state: State) -> AgentResponse:
        prompt = f'''
        You are a tutor in {state.subject} for a learner at the {state.educational_level} level.

        Give 1 question on {state.topic} and it's corresponding answer in JSON output.

        Output a JSON object with the keys question and answer
        '''

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{'role': 'user', 'content': prompt}],
            response_format={"type": "json_object"},
        )
        msg = resp.choices[0].message.content
        print('Quiz: ', msg)
        quiz = json.loads(msg)
        current_state = self.copy_state(state)
        current_state.quiz = quiz

        return AgentResponse(
            agent='🧠 QuizMaster',
            message=quiz.get('question'),
            current_state=current_state,
        )



class Learner(Agent):

    @override
    def run(self, state: State) -> AgentResponse:
        should_be_correct = random.random() < 0.5

        prompt = f'''
        You are a learner studying {state.subject} at the {state.educational_level} level.

        Give a only short {"correct" if should_be_correct else "incorrect" } answer to the question on {state.topic}:

        {state.quiz['question']}
        '''

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{'role': 'user', 'content': prompt}],
        )
        msg = resp.choices[0].message.content

        current_state = self.copy_state(state)
        current_state.learner_answer = msg

        return AgentResponse(
            agent='🧑 Learner',
            message=msg,
            current_state=current_state,
        )


class Reviewer(Agent):
    @override
    def run(self, state: State) -> AgentResponse:
        prompt = f'''
        You are an independent reviewer in {state.subject} at the {state.educational_level} level.

        The correct answer is: {state.quiz['answer']}
        The learner's answer is: {state.learner_answer}
        Rate the learner's answer between 0 to 100 with 100 being a perfect answer:

        Return only the rating value.
        '''

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{'role': 'user', 'content': prompt}],
        )
        rating = float(resp.choices[0].message.content)

        current_state = self.copy_state(state)

        return AgentResponse(
            agent='🔍 Reviewer',
            message=f'Learner scored {rating}% {'✅' if rating > 50 else '❌'}',
            current_state=current_state,
        )

### Workflow

In [10]:
class QuizWorkflow(Agent):
    def __init__(self):
        super().__init__()
        print('⚙️ Initializing QuizWorkflow')

        self.quiz_master = QuizMaster()
        self.learner = Learner()
        self.reviewer = Reviewer()


    @override
    def run(self, state: State) -> AgentResponse:
        current_state = self.copy_state(state)

        num_questions = state.num_questions_per_topic
        print(f'\n⚙️ Beginning the quiz workflow for {num_questions} questions\n')
        for i in range(num_questions):

            response = self.quiz_master.run(current_state)
            add_message(response.agent, response.message)
            current_state = response.current_state

            response = self.learner.run(current_state)
            add_message(response.agent, response.message)
            current_state = response.current_state

            response = self.reviewer.run(current_state)
            add_message(response.agent, response.message)
            current_state = response.current_state

        print(f'\n⚙️ Completed workflow')

        return AgentResponse(
            agent='🔍 QuizWorkflow',
            message=f'Done running workflow%',
            current_state=current_state,
        )


class Orchestrator(Agent):
    def __init__(self):
        super().__init__()
        print('⚙️ Initializing Orchestrator')

        self.curriculum_designer = CurriculumDesigner()
        self.tutor = Tutor()
        self.quiz_workflow = QuizWorkflow()


    @override
    def run(self, state: State) -> AgentResponse:
        end_time = datetime.now() + timedelta(minutes=state.run_duration_minutes)
        add_message('🤖 Orchestrator', f'Starting autonomous session on "{state.subject}" ({state.educational_level})')

        response = self.curriculum_designer.run(state=state)
        add_message(response.agent, response.message)

        state = response.current_state
        topics = state.topics

        for topic_id, topic in enumerate(topics, start=1):
            if datetime.now() >= end_time:
                add_message('⏰ Time', 's up! Ending session.')

            state.topic = topic
            print(f'📌 Topic {topic_id}/{len(topics)}: {topic} ---\n')

            response = self.tutor.run(state)
            add_message(response.agent, response.message)

            response = self.quiz_workflow.run(state)

            print('\n\n' + '*' * 80 + '\n\n')

        print('✅ Session complete! Final mastery:', state.mastery)


In [ ]:
def initialize_state() -> State:
    state = State(
        educational_level='Lower Primary',
        subject='Chemistry',
        mastery={},
        topic='',
        topics=[],
        history=[],
        quiz={'question': '', 'answer': ''},
        session_start = datetime.now()
    )

    return state

## Run

Bare-bones no UI as that is not the focus

In [9]:
# run_autonomous_study_session(duration_minutes=3)

orchestrator = Orchestrator()
state = initialize_state()
orchestrator.run(state)

⚙️ Initializing Orchestrator
⚙️ Initializing QuizWorkflow

[12:06:14] 🤖 Orchestrator: Starting autonomous session on "Chemistry" (Lower Primary)


[12:06:18] CurriculumDesigner: 5 topics. ['Atoms and Molecules', 'States of Matter', 'Properties of Materials', 'Basic Chemical Reactions', 'Introduction to Elements']

📌 Topic 1/5: Atoms and Molecules ---


[12:06:21] 📘 Tutor: Atoms are the tiniest pieces that make up everything around us. Think of them like tiny Lego bricks. Each atom has a center called a nucleus with positively charged “protons,” and a few neutrons, and little “electrons” orbit around the nucleus, like tiny balls spinning on invisible strings.  

When a few atoms connect, they form a molecule – a little family of atoms that stick together. Imagine two Lego bricks glued together to make a new shape. Water is a molecule made of two hydrogen atoms and one oxygen atom. In the same way, the air we breathe, the food we eat, and even the paper we write on are all made of countl

In [16]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def add_message(role: str, text: str):
    timestamp = datetime.now().strftime('%H:%M:%S')
    with output_area:
        print(f'\n[{timestamp}] {role}: {text}\n')
    time.sleep(1)  # optional


level = widgets.Dropdown(
    options=['Primary', 'Secondary', 'Undergraduate', 'Graduate'],
    description='Educational Level',
)

subject = widgets.Text(description='Subject', placeholder= 'Chemistry')
duration = widgets.IntSlider(
    value=3,
    min=2,
    max=30,
    step=1,
    description='Duration (min):',
    style={'description_width': 'initial'}
)

button = widgets.Button(description='Learn', icon='check')
ui = widgets.HBox(children=[subject, level, duration, button])

output_area = widgets.Output(
    layout={
        'width': '100%',
        'height': '400px',
        'overflow_y': 'auto',
        'border': '1px solid #ddd',
        'padding': '10px',
        'background': '#f9f9f9'
    }
)


def on_learn_click(b):
    session_level = level.value
    session_subject = subject.value
    session_duration = duration.value

    with output_area:
        if not session_subject:
            print('⚠️ Please enter a subject.')
            return

        print(f'🤖 Starting session on {session_subject} at {session_level} level')

        orchestrator = Orchestrator()
        state = initialize_state()
        state.educational_level = session_level
        state.subject = session_subject
        state.run_duration_minutes = session_duration

        orchestrator.run(state)

button.on_click(on_learn_click)

display(ui)
display(output_area)

Output(layout=Layout(border_bottom='1px solid #ddd', border_left='1px solid #ddd', border_right='1px solid #dd…